In [41]:
import numpy as np
import utility_b as U

Αλγόριθμος RANSAC

1. Επίλεξε τυχαία τον ελάχιστον αριθμό σημείων που απαιτοούνται για τον προσδιορισμό των παραμέτρων του μοντέλου.
2. Λύσε το πρόβλημα με τις παραμέτρους του μοντέλου.
3. Προσδιόρισε πόσα σημεία από το σύνολο όλων των σημείων ταιριάζουν με μια προκαθορισμένη ανοχή.
4. Εαν το κλάσμα του πλήθος των outliers δια το πλήθος όλων των σημείων υπερβαίνει ένα προκαθορισμένο όριο t, επίλεξε νέες παραμέτρους βάση των inliers.
5. Διαφορετικά, επανέλαβε τα βήματα 1 - 4 (το πολύ Ν φορές)

inliers -> σημεία του plane που θέλουμε να βρούμε ώστε να τα αφαιρέσουμε
outlier -> σημεία που δεν ανήκουν σε κάποιο plane

In [42]:
# load pointcloud
vertices = np.load('../pointcloud/gt_vertices.npy')
colors = np.load('../pointcloud/gt_colors.npy')

print(vertices.shape)
print(colors.shape)

(91126, 3)
(91126, 3)


In [43]:
# ===================== GRAOUND TROUTH POINTCLOUD =====================
n_iterations = 1200
# threashold for graound trouth will be 150
# threashold for my pointcloud will be 20

threshold_distance = 150  # Define a threshold distance to consider a point as an inlier

total_inlier_points = np.array([])
total_inlier_colors = np.array([])
total_outlier_points = np.array([])
total_outlier_colors = np.array([])

# Run the RANSAC algorithm

best_plane, best_inliers = U.ransac_plane_fit(vertices, n_iterations, threshold_distance)

# Access the best fit plane coefficients
# a, b, c, d = best_plane

# Access the inlier points from the point cloud
inlier_points = vertices[best_inliers]
inlier_colors = colors[best_inliers]
print("inlier points shape: ", inlier_points.shape)
print("inlier colors shape: ", inlier_colors.shape)

# Access the outlier points from the point cloud
outliers_indecies = np.delete(np.arange(vertices.shape[0]), best_inliers)
outlier_points = vertices[outliers_indecies]
outlier_colors = colors[outliers_indecies]
print("outlier points shape: ", outlier_points.shape)
print("outlier colors shape: ", outlier_colors.shape)

# append each point to total points
total_inlier_points = inlier_points
total_inlier_colors = inlier_colors
total_outlier_points = outlier_points
total_outlier_colors = outlier_colors

n_iterations = 1200
threshold_distance = 2000
threshold_color = 0.3
vertices = outlier_points
colors = outlier_colors
best_inliers = U.ransac_line_fit(vertices, colors, n_iterations, threshold_distance, threshold_color)

# Access the inlier points from the point cloud
inlier_points = vertices[best_inliers]
inlier_colors = colors[best_inliers]
print("inlier points shape: ", inlier_points.shape)
print("inlier colors shape: ", inlier_colors.shape)

# Access the outlier points from the point cloud
outliers_indecies = np.delete(np.arange(vertices.shape[0]), best_inliers)
outlier_points = vertices[outliers_indecies]
outlier_colors = colors[outliers_indecies]
print("outlier points shape: ", outlier_points.shape)
print("outlier colors shape: ", outlier_colors.shape)

total_inlier_points = np.vstack((total_inlier_points, inlier_points))
total_inlier_colors = np.vstack((total_inlier_colors, inlier_colors))
total_outlier_points = outlier_points
total_outlier_colors = outlier_colors

# save inlier points and colors
np.save('../pointcloud/gt_planes.npy', total_inlier_points)
np.save('../pointcloud/gt_planes_colors.npy', total_inlier_colors)

# save outlier points and colors
np.save('../pointcloud/gt_objects.npy', total_outlier_points)
np.save('../pointcloud/gt_objects_colors.npy', total_outlier_colors)


inlier points shape:  (35905, 3)
inlier colors shape:  (35905, 3)
outlier points shape:  (55221, 3)
outlier colors shape:  (55221, 3)
inlier points shape:  (7290, 3)
inlier colors shape:  (7290, 3)
outlier points shape:  (47931, 3)
outlier colors shape:  (47931, 3)


In [49]:
# load my pointcloud
vertices = np.load('../pointcloud/vertices.npy')
colors = np.load('../pointcloud/colors.npy')

print(vertices.shape)
print(colors.shape)

(158445, 3)
(158445, 3)


In [50]:
# ===================== MY POINTCLOUD =====================

# wall plane
n_iterations = 1200

threshold_distance = 35  # Define a threshold distance to consider a point as an inlier

total_inlier_points = []
total_inlier_colors = []


# Run the RANSAC algorithm
best_plane, best_inliers = U.ransac_plane_fit(vertices, n_iterations, threshold_distance)

# Access the inlier points from the point cloud
inlier_points = vertices[best_inliers]
inlier_colors = colors[best_inliers]
print("inlier points shape: ", inlier_points.shape)
print("inlier colors shape: ", inlier_colors.shape)


total_inlier_points = inlier_points
total_inlier_colors = inlier_colors


# Access the outlier points from the point cloud
outliers_indecies = np.delete(np.arange(vertices.shape[0]), best_inliers)
outlier_points = vertices[outliers_indecies]
outlier_colors = colors[outliers_indecies]
print("outlier points shape: ", outlier_points.shape)
print("outlier colors shape: ", outlier_colors.shape)


n_iterations = 50
threshold_distance = 200
color_threshold_distance = 0.6
neighbors_inlier_indicies, outlier_points, outlier_colors = U.neighbors_inlier(total_inlier_points,outlier_points,total_inlier_colors, outlier_colors, n_iterations, threshold_distance, color_threshold_distance)
print("neighbors_inlier_indicies shape: ", neighbors_inlier_indicies.shape)

# append each point to total points
neighbors_inlier = outlier_points[neighbors_inlier_indicies]
neighbors_inlier_colors = outlier_colors[neighbors_inlier_indicies]

# # trasnfom best inlier color to red 
# inlier_colors[:,0] = 1
# inlier_colors[:,1] = 0
# inlier_colors[:,2] = 0

# # transform neightbours inlier color to green
# neighbors_inlier_colors[:,0] = 0
# neighbors_inlier_colors[:,1] = 1
# neighbors_inlier_colors[:,2] = 0

total_inlier_points = np.vstack((total_inlier_points, neighbors_inlier))
total_inlier_colors = np.vstack((total_inlier_colors, neighbors_inlier_colors))

outlier_points = np.delete(outlier_points, neighbors_inlier_indicies, axis=0)
outlier_colors = np.delete(outlier_colors, neighbors_inlier_indicies, axis=0)


# road plane
# threshold_distance = 10
# n_iterations = 1200
# vertices = outlier_points
# best_plane, best_inliers = U.ransac_plane_fit(vertices, n_iterations, threshold_distance)

# # Access the inlier points from the point cloud
# inlier_points = vertices[best_inliers]
# inlier_colors = colors[best_inliers]
# print("inlier points shape: ", inlier_points.shape)
# print("inlier colors shape: ", inlier_colors.shape)

# total_inlier_points = np.vstack((total_inlier_points, inlier_points))
# total_inlier_colors = np.vstack((total_inlier_colors, inlier_colors))

# outlier_points = np.delete(outlier_points, best_inliers, axis=0)
# outlier_colors = np.delete(outlier_colors, best_inliers, axis=0)

# # save inlier points and colors
np.save('../pointcloud/my_planes.npy', total_inlier_points)
np.save('../pointcloud/my_planes_colors.npy', total_inlier_colors)

# save outlier points and colors
np.save('../pointcloud/my_objects.npy', outlier_points)
np.save('../pointcloud/my_objects_colors.npy', outlier_colors)

/Users/johnefthymiou/Documents/university/8th_semester/3D/3D_Scene_Segmentation_and_Detection/part_b/utility_b.py:56: RuntimeWarning: divide by zero encountered in double_scalars
  slope3 = (points[2, 1] - points[0, 1]) / (points[2, 0] - points[0, 0])


inlier points shape:  (51054, 3)
inlier colors shape:  (51054, 3)
outlier points shape:  (107391, 3)
outlier colors shape:  (107391, 3)
possible neighbor inlier points after color comparison: (89638,)
neighbors_inlier_indicies shape:  (2829,)
